In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-11-29 03:01:30--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.04s   

2023-11-29 03:01:30 (27.2 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [3]:
#read the file
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [4]:
#Printing the length of dataset in characters
print("Length of dataset in characters: ", len(text))

Length of dataset in characters:  1115394


In [5]:
#Printing the first 1000 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [7]:
# Get unique chars as a sorted list in the dataset
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [15]:
#Char tokenizer
# Other tokenziers = SentencePiece = subword
# Tiktoken = gpt

stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("Hello world!"))
print(decode(encode("Hello world!")))

[20, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42, 2]
Hello world!


In [16]:
#Tiktoken example
import tiktoken
enc = tiktoken.get_encoding('gpt2')
print(enc.n_vocab) #Vocab size
print(enc.encode("Hello world!"))
print(enc.decode(enc.encode("Hello world!")))

50257
[15496, 995, 0]
Hello world!


In [18]:
#Encoding entire dataset and storing as a tensor
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # Printing the first 1000 encoded characters of the dataset

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [19]:
# Splitting into train and validation set
n = int(0.9 * len(data))
train_data = data[:n]
valid_data = data[n:]

In [20]:
block_size = 8 #Chunk of the data set i.e it has 8 examples packed
train_data[:block_size+1] # Ex if our input input is 18 47 56, we are guessing the next character 58
# The other reason we use block_size is such that the transformer network is used to 
# see contexts as little as 1 upto block_size
# These chunks are time dimension. The other dimension is the batch dimension -> effiency to ensure gpus are busy 

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [23]:
x = train_data[:block_size]
y = train_data[1:block_size + 1]
for t in range(block_size):
    context = x[:t + 1]
    target = y[t]
    print (f"When input is {context} the target: {target}")

When input is tensor([18]) the target: 47
When input is tensor([18, 47]) the target: 56
When input is tensor([18, 47, 56]) the target: 57
When input is tensor([18, 47, 56, 57]) the target: 58
When input is tensor([18, 47, 56, 57, 58]) the target: 1
When input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
When input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [30]:
torch.manual_seed(1337)
batch_size = 4 # How many independent sequences will we process in parallel?
block_size = 8 # What is the maximum context length for predictions?

def get_batch(split):
    #Generate small batches of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,)) #Generate batchsize number of random offsets
    x = torch.stack([data[i:i + block_size] for i in ix])
    y = torch.stack([data[i + 1:i + block_size + 1] for i in ix])

    return x, y

xb, yb = get_batch("train")
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print("----")

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b,:t+1]
        target = yb[b, t]
        print(f"When input is {context.tolist()} the target: {target}")


inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
When input is [24] the target: 43
When input is [24, 43] the target: 58
When input is [24, 43, 58] the target: 5
When input is [24, 43, 58, 5] the target: 57
When input is [24, 43, 58, 5, 57] the target: 1
When input is [24, 43, 58, 5, 57, 1] the target: 46
When input is [24, 43, 58, 5, 57, 1, 46] the target: 43
When input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
When input is [44] the target: 53
When input is [44, 53] the target: 56
When input is [44, 53, 56] the target: 1
When input is [44, 53, 56, 1] the target: 58
When input is [44, 53, 56, 1, 58] the target: 46
When input is [44, 53

In [31]:
print(xb) # Our batch of input to transformer

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [45]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # Each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and tagets are both (B, T) tensor of integers
        logits = self.token_embedding_table(idx) # (B, T, C) For each value in idx we index the table and get that row
        # For example for 24, we get the 24th row in the embedding table. (B, T, C) = (4, 8, 64)
        # Logits are the score for the next character in the sequence. We are predicting what comes next based on
        # individual identity of a single token (not correlating other characters), 
        # because some characters follow other characters.

        if(targets == None):
            loss = None
        else:    
            B, T, C = logits.shape
            # Reshape for cross_entropy function
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)
        
        
        return logits, loss

    # The job of generate is to take idx (B, T) to generate (B, T + 1) .. upto max_new_tokens (B, T +  max_new_tokens)
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # Get the predictions
            logits, loss = self(idx)
            # Focus only on the last time step
            logits = logits [:, -1, :] # Becomes (B, C)
            # Apply softmax to get probabilities
            probs = F.softmax(logits, dim=1) # (B, C)
            # Sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, C)
            # Append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T + 1)
        
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss) # Loss to be around -ln(1/65) = 4.174
# Generate 100 tokens given new line char i.e (B, T) = (1, 1) = 0
print(decode(m.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [46]:
# Create pytorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3) # 1e-4 for big models, since our model is small we are using 1e-3

In [53]:
batch_size = 32
for steps in range(10000):
    #Sample a batch of data
    xb, yb = get_batch('train')
    
    #Evaluate loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.3514575958251953


In [56]:
print(decode(m.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=400)[0].tolist()))


CATMIS: wibacit e m moorn me bathendwhot
DWhol, 'd be f pard h dimak, Cale rodes I ferdou.
Whay;
ORKid nd apands h t
Wh htthele agasharnde g woa Whicoy bree ou t whoronietomes BREno-
BO:
Vity, thinuthayove cu.
OF m D wokeacan igoprer menes bawotorch, hy nchthathaghie gemed s m.
KIOfedo be
QUKimofut henuth til;
Thed tht te rp wod th sthulitharm aullou, INED:
An,


Limon:
Nofot tian GA s g pr thy br


# The mathematical trick in self-attention

In [57]:
# Consider the following toy example
# So far the tokens were not talking to each other (i.e T) since it was a bigram model
# Now for attention, the token in 5 for example should not communicate with token in 6, 7, 8 location
# A simple way to do that would be to get the channels (logits) of 5, 4, 3, 2, 1 and average them
# to get a good feature vector which will have the correlation of the previous characters
# The average would be lossy and since we lost a ton of information on the spatial arrangements of tokens
torch.manual_seed(1337)
B, T, C = 4, 8, 2 # batch, time, channels
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [71]:
# We want x[b, t] = mean_{i <= t} x[b, i]
# x bag of words (averaging the bag of words)
# Version 1
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t, C)
        # Avering in 0th dimension i.e t
        xbow[b,t] = torch.mean(xprev, 0) # C

In [72]:
# Vectorizing the computation using the trick below
# Version 2
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (T, T) x (B, T, C) ----> (B, T, T) x (B, T, C) ----> (B, T, C)
#it will create the batch dimension and multiply each bach element in parallel so we do (T,  T) x (T, C)

torch.allclose(xbow, xbow2) # Verifies our method

True

In [73]:
# Version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
# zeros will indicate what tokens will communicate with others
# In future this wont be all zeros for 
wei = torch.zeros((T, T))
# -inf indicates the token that wont communicate
wei = wei.masked_fill(tril == 0, float('-inf')) # Fills 0 of wei with -inf
# Softmax -> exponentiate each of the elements and divide by sum
wei = F.softmax(wei, dim=-1) # Softmax on each row, its like a normalization
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

True

In [61]:
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [62]:
xbow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [65]:
# The math trick
# using torch.tril on torch.ones, we get sum like the example above
# To do average, we normalize each row of torch.tril such that they add to one

In [69]:
torch.manual_seed(42)
a = torch.tril(torch.ones((3,3)))
a = a / torch.sum(a, 1, keepdim=True) # Normalize to get the result C as average
b = torch.randint(0, 10, (3,2)).float()
c = a @ b # Matrix multiplication of torch
print('a=')
print(a)
print('b=')
print(b)
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])
